# 1、字符串输出解析器 StrOutputParser

In [6]:
# 1、获取大模型
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser, XMLOutputParser

import os
import dotenv
from langchain_core.utils import pre_init
from langchain_openai import ChatOpenAI

dotenv.load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("LLM_API_KEY")
os.environ['OPENAI_BASE_URL'] = os.getenv("LLM_BASE_URL")

chat_model = ChatOpenAI(model=os.getenv("LLM_MODEL_ID"))

# 2、调用大模型
response = chat_model.invoke("什么是大语言模型？")
# print(type(response))   # AIMessage

# 3、如何获取一个字符串的输出结果呢？
# # 方式1：自己调用输出结果的content
# print(response.content)

# 方式2：使用StrOutputParser
parser = StrOutputParser()
str_response = parser.invoke(response)
print(type(str_response))  # <class 'str'>
print(str_response)


<class 'str'>
大语言模型是指一种能够生成自然语言文本的机器学习模型，这些模型通常基于深层神经网络结构，如变换器（Transformer）架构，具有大量的参数（通常超过百万级甚至亿级参数）。大语言模型通过对大量文本数据进行训练，可以理解和生成与人类相似的自然语言文本，具备强大的语言理解和生成能力。

大语言模型能够完成的任务范围广泛，包括但不限于文本补全、机器翻译、情感分析、问答系统、文本分类、摘要生成等。在近年来自然语言处理领域取得了重要进展，成为诸多文本生成任务的有力工具。代表性的大语言模型包括但不限于：阿里云的通义千问、谷歌的LaMDA、百度的文心一言、阿里云的通义万相、百度ERNIE等。


# 2、JsonOutputParser : Json输出解析器

方式1：

In [8]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate

chat_model = ChatOpenAI(model=os.getenv("LLM_MODEL_ID"))

chat_prompt_template = ChatPromptTemplate.from_messages([
    ("system", "你是一个靠谱的{role}"),
    ("human", "{question}")
])

# 正确的：
prompt = chat_prompt_template.invoke(
    input={"role": "人工智能专家", "question": "人工智能用英文怎么说？问题用q表示，答案用a表示，返回一个JSON格式的数据"})

# 错误的：
# prompt = chat_prompt_template.invoke(input={"role":"人工智能专家","question":"人工智能用英文怎么说？"})

response = chat_model.invoke(prompt)
# print(response.content)

# 获取一个JsonOutputParser的实例
parser = JsonOutputParser()

json_result = parser.invoke(response)
print(json_result)

{'q': '人工智能用英文怎么说？', 'a': '人工智能用英文说为Artificial Intelligence (AI)'}


方式2：

举例1：

In [9]:
parser = JsonOutputParser()

print(parser.get_format_instructions())

Return a JSON object.


举例2：

In [10]:
# 引入依赖包
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate

# 初始化语言模型
chat_model = ChatOpenAI(model=os.getenv("LLM_MODEL_ID"))

joke_query = "告诉我一个笑话。"

# 定义Json解析器
parser = JsonOutputParser()

# 以PromptTemplate为例
prompt_template = PromptTemplate.from_template(
    template="回答用户的查询\n 满足的格式为{format_instructions}\n 问题为{question}\n",
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

prompt = prompt_template.invoke(input={"question": joke_query})
response = chat_model.invoke(prompt)
print(response)

json_result = parser.invoke(response)
print(json_result)

content='```json\n{\n  "joke": "为什么电脑经常生病？因为窗户（Windows）总是开着！"\n}\n```' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 52, 'total_tokens': 78, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'Qwen/Qwen2.5-7B-Instruct', 'system_fingerprint': '', 'id': '019bf9036ae9f033638cd9c64685e96b', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--019bf903-6d88-71c3-8ff1-7690fd2a1514-0' tool_calls=[] invalid_tool_calls=[] usage_metadata={'input_tokens': 52, 'output_tokens': 26, 'total_tokens': 78, 'input_token_details': {}, 'output_token_details': {}}
{'joke': '为什么电脑经常生病？因为窗户（Windows）总是开着！'}


知识的拓展： |

In [11]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate

chat_model = ChatOpenAI(model=os.getenv("LLM_MODEL_ID"))

chat_prompt_template = ChatPromptTemplate.from_messages([
    ("system", "你是一个靠谱的{role}"),
    ("human", "{question}")
])

# 获取一个JsonOutputParser的实例
parser = JsonOutputParser()

# 写法1：
# prompt = chat_prompt_template.invoke(input={"role":"人工智能专家","question":"人工智能用英文怎么说？问题用q表示，答案用a表示，返回一个JSON格式的数据"})
#
# response = chat_model.invoke(prompt)
#
# json_result = parser.invoke(response)
# print(json_result)

# 写法2：
chain = chat_prompt_template | chat_model | parser
json_result1 = chain.invoke(
    input={"role": "人工智能专家", "question": "人工智能用英文怎么说？问题用q表示，答案用a表示，返回一个JSON格式的数据"}
    )
print(json_result1)

{'q': '人工智能用英文怎么说？', 'a': 'Artificial Intelligence (AI)'}


针对于举例2

In [14]:
# 引入依赖包
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate

# 初始化语言模型
chat_model = ChatOpenAI(model=os.getenv("LLM_MODEL_ID"))

joke_query = "告诉我一个笑话。"

# 定义Json解析器
parser = JsonOutputParser()

# 以PromptTemplate为例
prompt_template = PromptTemplate.from_template(
    template="回答用户的查询\n 满足的格式为{format_instructions}\n 问题为{question}\n",
    partial_variables={"format_instructions": parser.get_format_instructions()},
)
# 写法1：
# prompt = prompt_template.invoke(input={"question":joke_query})
# response = chat_model.invoke(prompt)
# json_result = parser.invoke(response)

chain = prompt_template | chat_model | parser
json_result = chain.invoke(input={"question": joke_query})
print(json_result)

{'joke': '为什么电脑经常生病？因为窗户（windings）总是开着！'}
